In [3]:
import pandas as pd
from sqlalchemy import create_engine, inspect
import json
pd.set_option('display.max_columns',None)

In [4]:
df = pd.read_csv("downloads/executions.csv")
df.head()

,Date,Name,Age,Sex,Race,Crime,Victim Count,Victim Sex,Victim Race,County,State,Region,Method,Juvenile,Volunteer,Federal,Foreign National
0,01/17/1977,Gary Gilmore,36,Male,White,Murder,1,Male,White,Utah,UT,West,Firing Squad,No,Yes,No,No
1,05/25/1979,John Spenkelink,30,Male,White,Murder,1,Male,White,Leon,FL,South,Electrocution,No,No,No,No
2,10/22/1979,Jesse Bishop,46,Male,White,Murder,1,Male,White,Clark,NV,West,Gas Chamber,No,Yes,No,No
3,03/09/1981,Steven Judy,24,Male,White,Murder,4,"2 Male, 2 Female",White,Marion,IN,Midwest,Electrocution,No,Yes,No,No
4,08/10/1982,Frank Coppola,38,Male,White,Murder,1,Male,White,Newport News City,VA,South,Electrocution,No,Yes,No,No


In [5]:
df['Date'] = pd.to_datetime(df['Date'])

In [6]:
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month 

In [7]:
df['County & State'] = df['County'].astype(str) +", "+ df["State"]

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1442 entries, 0 to 1441
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              1442 non-null   datetime64[ns]
 1   Name              1442 non-null   object        
 2   Age               1442 non-null   int64         
 3   Sex               1442 non-null   object        
 4   Race              1442 non-null   object        
 5   Crime             1442 non-null   object        
 6   Victim Count      1442 non-null   int64         
 7   Victim Sex        1437 non-null   object        
 8   Victim Race       1436 non-null   object        
 9   County            1440 non-null   object        
 10  State             1442 non-null   object        
 11  Region            1442 non-null   object        
 12  Method            1442 non-null   object        
 13  Juvenile          1442 non-null   object        
 14  Volunteer         1442 n

In [9]:
df.head()

,Date,Name,Age,Sex,Race,Crime,Victim Count,Victim Sex,Victim Race,County,State,Region,Method,Juvenile,Volunteer,Federal,Foreign National,Year,Month,County & State
0,1977-01-17,Gary Gilmore,36,Male,White,Murder,1,Male,White,Utah,UT,West,Firing Squad,No,Yes,No,No,1977,1,"Utah, UT"
1,1979-05-25,John Spenkelink,30,Male,White,Murder,1,Male,White,Leon,FL,South,Electrocution,No,No,No,No,1979,5,"Leon, FL"
2,1979-10-22,Jesse Bishop,46,Male,White,Murder,1,Male,White,Clark,NV,West,Gas Chamber,No,Yes,No,No,1979,10,"Clark, NV"
3,1981-03-09,Steven Judy,24,Male,White,Murder,4,"2 Male, 2 Female",White,Marion,IN,Midwest,Electrocution,No,Yes,No,No,1981,3,"Marion, IN"
4,1982-08-10,Frank Coppola,38,Male,White,Murder,1,Male,White,Newport News City,VA,South,Electrocution,No,Yes,No,No,1982,8,"Newport News City, VA"


In [10]:
dfc = pd.read_excel('Downloads/usa_county.xlsx')

In [11]:
dfc['County & State'] = dfc['County'].astype(str) +", "+ dfc["State"]

In [12]:
dfc.head()

,County,State,Coordinates,County & State
0,Wrangell,AK,"41.001938375, -83.6665354077","Wrangell, AK"
1,Aleutians East,AK,"38.4207190191, -77.4574003866","Aleutians East, AK"
2,Aleutians West,AK,"40.1764332141, -98.4999473753","Aleutians West, AK"
3,Denali,AK,"28.4225359982, -99.756726125","Denali, AK"
4,North Slope,AK,"41.7723266607, -91.1324230468","North Slope, AK"


In [13]:
df2 = pd.merge(df, dfc, on="County & State")

In [14]:
df2.columns

Index(['Date', 'Name', 'Age', 'Sex', 'Race', 'Crime', 'Victim Count',
       'Victim Sex', 'Victim Race', 'County_x', 'State_x', 'Region', 'Method',
       'Juvenile', 'Volunteer', 'Federal', 'Foreign National', 'Year', 'Month',
       'County & State', 'County_y', 'State_y', 'Coordinates'],
      dtype='object')

In [15]:
df3 = df2.loc[:,['Date', 'Month', 'Year', 'Name', 'Age', 'Sex', 'Race', 'Victim Count',  'Method', 'County_x', 'State_x', 'Coordinates']]

In [16]:
df3 = df3.rename(columns={"County_x": "County", "State_x": "State"})

In [17]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1422 entries, 0 to 1421
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Date          1422 non-null   datetime64[ns]
 1   Month         1422 non-null   int64         
 2   Year          1422 non-null   int64         
 3   Name          1422 non-null   object        
 4   Age           1422 non-null   int64         
 5   Sex           1422 non-null   object        
 6   Race          1422 non-null   object        
 7   Victim Count  1422 non-null   int64         
 8   Method        1422 non-null   object        
 9   County        1422 non-null   object        
 10  State         1422 non-null   object        
 11  Coordinates   1422 non-null   object        
dtypes: datetime64[ns](1), int64(4), object(7)
memory usage: 144.4+ KB


In [20]:
path = "sqlite:///executions.sqlite"
engine = create_engine(path)

In [21]:
df3.to_sql("executions", con=engine, method="multi", if_exists="replace", index=False)

1422

In [22]:
# INSPECT
inspector_gadget = inspect(engine)

# get all tables
table_names = inspector_gadget.get_table_names()
for table in table_names:
    print(table)
    
    # get columns for table
    for column in inspector_gadget.get_columns(table):
        print(column)
        
    print()

executions
{'name': 'Date', 'type': DATETIME(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'Month', 'type': BIGINT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'Year', 'type': BIGINT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'Name', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'Age', 'type': BIGINT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'Sex', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'Race', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'Victim Count', 'type': BIGINT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'Method', 'type': TEXT(), 'nullable': True, 'default': None, 'autoincreme

In [23]:
#df3.to_json('executions_updated.json', orient='records', lines=True)

In [24]:
df3.to_json(r'executions_updated4', orient ='records', indent = 2)